In [1]:
#include <iostream>

/*a workaround to solve cling issue*/
#include "../inc/macos_cling_workaround.hpp"
/*set libtorch path, load libs*/
#include "../inc/load_libtorch.hpp"
/*import custom defined macros*/
#include "../inc/custom_def.hpp"
/*import matplotlibcpp*/
#include "../inc/load_matplotlibcpp.hpp"

/*import libtorch header file*/
#include <torch/torch.h>

# 线性回归的简单实现：使用libtorch API

In [2]:
constexpr int num_example = 500;
constexpr int batch_size = 10;

torch::Tensor true_w = torch::tensor(at::ArrayRef<float>({2.0, -3.4}));
torch::Tensor true_b = torch::tensor(at::ArrayRef<float>({4.2}));

torch::Tensor X = torch::normal(0.0, 1.0, {num_example, true_w.size(0)}); 
torch::Tensor y = X.matmul(true_w) + true_b;
y = y + torch::normal(0.0, 0.1, y.sizes());
y = torch::reshape(y, {num_example, 1});

torch::Tensor train_data = torch::cat({X, y}, 1);
// printT(train_data);

auto data_set = torch::data::datasets::TensorDataset(train_data);
auto data_loader = torch::data::make_data_loader<torch::data::samplers::RandomSampler>
                    (std::move(data_set), batch_size);

In [3]:
std::tuple<torch::Tensor, torch::Tensor> cat_batch(std::vector<torch::data::Example<at::Tensor, void>> &e, int batch_size)
{
    torch::Tensor data  = torch::zeros({batch_size, true_w.size(0)});
    torch::Tensor label = torch::zeros({batch_size, true_b.size(0)});
    
    for (int i = 0; i < batch_size; i++) {
        auto t = e[i].data;
        for (int j = 0; j < true_w.size(0); j++) {
            data[i][j] = t[j];
        }
        label[i]   = t[true_w.size(0)];
    }
        
    return std::make_tuple(data, label);
}

int idx = 0;
for (auto& batch : *data_loader) {
    /**
     *   here in our test case,
     *   batch is `std::vector<torch::data::Example<at::Tensor, void>>`
     *   notice:
     *   here for this torch::data::Example, we only have data, no target
     *
     */
    idx++;
//     auto raw = batch[0].data;
//     printT(raw);
    auto sample_t = cat_batch(batch, batch_size);
    if (idx == 2) {
        printT(std::get<0>(sample_t));
        printT(std::get<1>(sample_t));
    }
}

std::get<0>(sample_t) = 
 0.9136 -1.3414
-0.6682  0.2977
 0.2865  0.9406
 2.0507 -0.1301
-1.2409 -1.5614
 1.3365 -0.3666
-1.8163  1.4520
 0.8634  0.9690
-2.2789  1.5065
-0.6701 -2.5495
[ CPUFloatType{10,2} ]
<<--->>

std::get<1>(sample_t) = 
 10.5357
  1.8761
  1.6894
  8.8734
  7.0857
  8.1169
 -4.3689
  2.4858
 -5.3991
 11.6323
[ CPUFloatType{10,1} ]
<<--->>



In [5]:
/**
 *   ref libtorch official tutorial
 *   (https://pytorch.org/cppdocs/frontend.html)
 */

// Define a new Module.
struct Net : torch::nn::Module {
  Net() {
    // Construct and register two Linear submodules.
    fc1 = register_module("fc1", torch::nn::Linear(2, 1));
  }

  // Implement the Net's algorithm.
  torch::Tensor forward(torch::Tensor x) {
    x = fc1->forward(x);
    return x;
  }

  torch::nn::Linear fc1{nullptr};
};

In [6]:
auto net = std::make_shared<Net>();
torch::optim::SGD optimizer(net->parameters(), /*lr=*/0.01);

In [9]:
for (size_t epoch = 1; epoch <= 10; ++epoch) {
    size_t batch_index = 0;
    // Iterate the data loader to yield batches from the dataset.
    for (auto& batch : *data_loader) {
        auto sample_t = cat_batch(batch, batch_size);
        auto data   = std::get<0>(sample_t);
        auto target = std::get<1>(sample_t);
        // Reset gradients.
        optimizer.zero_grad();
        // Execute the model on the input data.
        torch::Tensor prediction = net->forward(data);
        // Compute a loss value to judge the prediction of our model.
        torch::Tensor loss = torch::mse_loss(prediction, target);
        // Compute gradients of the loss w.r.t. the parameters of our model.
        loss.backward();
        // Update the parameters based on the calculated gradients.
        optimizer.step();
        // Output the loss and checkpoint every 100 batches.
        if (++batch_index % 10 == 0) {
        std::cout << "Epoch: " << epoch << " | Batch: " << batch_index
                  << " | Loss: " << loss.item<float>() << std::endl;
        // Serialize your model periodically as a checkpoint.
        //torch::save(net, "net.pt");
        }
    }
}

Epoch: 1 | Batch: 10 | Loss: 0.223734
Epoch: 1 | Batch: 20 | Loss: 0.232118
Epoch: 1 | Batch: 30 | Loss: 0.408283
Epoch: 1 | Batch: 40 | Loss: 0.125268
Epoch: 1 | Batch: 50 | Loss: 0.155616
Epoch: 2 | Batch: 10 | Loss: 0.0488156
Epoch: 2 | Batch: 20 | Loss: 0.0696044
Epoch: 2 | Batch: 30 | Loss: 0.0156382
Epoch: 2 | Batch: 40 | Loss: 0.0286981
Epoch: 2 | Batch: 50 | Loss: 0.0130505
Epoch: 3 | Batch: 10 | Loss: 0.0136231
Epoch: 3 | Batch: 20 | Loss: 0.0115919
Epoch: 3 | Batch: 30 | Loss: 0.00965719
Epoch: 3 | Batch: 40 | Loss: 0.00685044
Epoch: 3 | Batch: 50 | Loss: 0.0116952
Epoch: 4 | Batch: 10 | Loss: 0.0134756
Epoch: 4 | Batch: 20 | Loss: 0.00559405
Epoch: 4 | Batch: 30 | Loss: 0.00240321
Epoch: 4 | Batch: 40 | Loss: 0.00883842
Epoch: 4 | Batch: 50 | Loss: 0.0038392
Epoch: 5 | Batch: 10 | Loss: 0.0163589
Epoch: 5 | Batch: 20 | Loss: 0.0136825
Epoch: 5 | Batch: 30 | Loss: 0.0116613
Epoch: 5 | Batch: 40 | Loss: 0.008173
Epoch: 5 | Batch: 50 | Loss: 0.0147786
Epoch: 6 | Batch: 10 | Los

In [10]:
printT(net->parameters());

net->parameters() = 
 1.9995 -3.3978
[ CPUFloatType{1,2} ]  4.2011
[ CPUFloatType{1} ]
<<--->>

